### Step 2.2: Data Engineering


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

Load the data to df, see its shape and columns.

In [197]:
df = pd.read_csv('data_the_babe/df_1m_secure.csv')

In [198]:
df.shape  

(963592, 22)

In [199]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'HCost', '_id', 'adOption', 'cohort',
       'daysSinceInstall', 'hasEnoughCoin', 'hcl', 'levelMode', 'levelNumber',
       'numFails', 'offerReason', 'percentageLeft', 'platform', 'playTime',
       'playerId', 'result', 'segments', 'sessions', 'ts', 'twentileLeft'],
      dtype='object')

First things first, hash ***playerId*** because trust is everything.

- Created a function to hash input x
- Ran the function on each row of df.playerid

In [200]:
import hashlib

def hash_id(x):
    hash_object = hashlib.md5(x.encode())
    return hash_object.hexdigest()
    
df['playerId'] = df['playerId'].map(hash_id)

In [201]:
df.playerId.value_counts().head()  # it worked, looks like hash lib assigns same hash to same strings.

5a03d0c989ac120f2bb5df5a40046657    976
2ab3317d81139dda6b1dfc40ac39918c    578
25338d146532a09ab0f8b2ecb6ae25e8    475
f85e2c0ae4c8c96b0a60b8751c7798ef    402
e03ccfac932729338ee5845d361a93d3    397
Name: playerId, dtype: int64

Get rid of some of the useless columns.

In [202]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [203]:
df.drop(['twentileLeft'], axis=1, inplace = True)

In [204]:
df.head(1)

,Unnamed: 0.1,HCost,_id,adOption,cohort,daysSinceInstall,hasEnoughCoin,hcl,levelMode,levelNumber,numFails,offerReason,percentageLeft,platform,playTime,playerId,result,segments,sessions,ts
0,0,4,{'$oid': '58c80b93c1a7ac04b0a08f3b'},0,"{'day': 424.0, 'week': 60.0}",10.0,1,"{""Casual"": 162, ""alternative_game_mode"": 65}",Casual,163,2,Percentage,0,IOS,19,8727ec8a1aef99afbbb9cddc5a889b44,reject,"{'IAP': 'false', 'ISG_PBS_Price': 'Low', 'ISG_...",25,{'$date': {'$numberLong': '1489505171825'}}


Normally I would recommend to fill NA values with mean-median-min-max or something relevant. Luckyly in our case the proportion of nan values are so small that, dropping them will not negatively effect the future analisis.

***drops*** rows that include NA

In [205]:
df = df.dropna()

***Target*** (result)  I will be generating couple of targets for different senarios. Because I am not sure how to make more money for the company yet. In other words I need to see which senerio leads to most increase of revenue and which target receives most signal from features.

- 1 consider adwatch and bough as win
- Only consider bought as win
- Only consider adwatch as win

I recently do not know which target to focus on for best results. My assumption is that I shall focus merely on bought(coin spending) or merged target.



In [206]:
df.result.head(2)

0    reject
1    reject
Name: result, dtype: object

In [207]:
df['target_both'] = (df.result == "bought") | (df.result == "adWatch")
df['target_adwatch'] = (df.result == 'adWatch')
df['target_bought'] = (df.result == 'bought')
df.drop(['result'], axis = 1, inplace = True)

In [208]:
df.columns

Index(['Unnamed: 0.1', 'HCost', '_id', 'adOption', 'cohort',
       'daysSinceInstall', 'hasEnoughCoin', 'hcl', 'levelMode', 'levelNumber',
       'numFails', 'offerReason', 'percentageLeft', 'platform', 'playTime',
       'playerId', 'segments', 'sessions', 'ts', 'target_both',
       'target_adwatch', 'target_bought'],
      dtype='object')

***HCost***

Following function creates hot dummy variables for hot features.

In [209]:
def one_hot_dummy(input_df, columns):
    df_hot = input_df.copy()

    for col in columns:
        dummies = pd.get_dummies(df_hot[col])
#         dummies.drop(dummies.columns[-1], axis=1, inplace=True)  --> not my tempo
        df_hot = df_hot.drop(col, axis=1).merge(dummies, left_index=True, right_index=True)
    
    return df_hot

In [210]:
df = one_hot_dummy(df,['HCost'])

In [211]:
df.head(1)

,Unnamed: 0.1,_id,adOption,cohort,daysSinceInstall,hasEnoughCoin,hcl,levelMode,levelNumber,numFails,...,playerId,segments,sessions,ts,target_both,target_adwatch,target_bought,4,5,6
0,0,{'$oid': '58c80b93c1a7ac04b0a08f3b'},0,"{'day': 424.0, 'week': 60.0}",10.0,1,"{""Casual"": 162, ""alternative_game_mode"": 65}",Casual,163,2,...,8727ec8a1aef99afbbb9cddc5a889b44,"{'IAP': 'false', 'ISG_PBS_Price': 'Low', 'ISG_...",25,{'$date': {'$numberLong': '1489505171825'}},False,False,False,1,0,0


***_id*** --> get rid of it

In [212]:
df.drop(['_id'], axis = 1, inplace = True)

***adOption***  --> bool, let it stay as bool

***cohort*** --> set a range for every 10 weeks and, dummy them

convert_str_to_dict: converts strings to dictionary by opening and loading them on json format.

In [213]:
#use this one with all dictionary columns at later stage
import json

def convert_str_to_dict(x):
    json_acceptable_string = x.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    return d

In [214]:
df['cohort'] = df['cohort'].map(convert_str_to_dict)

In [215]:
df['cohort'][0]['week']

60.0

In [216]:
def weeks(x):
    return x['week']

df['cohort'] = df['cohort'].map(weeks)

In [217]:
df["cohort"] = df["cohort"].astype(float)

In [218]:
def week_group_cohort(x):

    if x <= 10:
        return 'w0'
    elif 10 < x <= 20:
        return 'w10'
    elif  20 < x <= 30:
        return 'w20'
    elif  30 < x <= 40:
        return 'w30'
    elif  40 < x <= 50:
        return 'w40'
    elif  50 < x <= 60:
        return 'w50'
    else:
        return 'w60'

df['cohort'] = df['cohort'].map(week_group_cohort)
        
    

In [219]:
df = one_hot_dummy(df,['cohort'])

In [220]:
df.head(1)

,Unnamed: 0.1,adOption,daysSinceInstall,hasEnoughCoin,hcl,levelMode,levelNumber,numFails,offerReason,percentageLeft,...,4,5,6,w0,w10,w20,w30,w40,w50,w60
0,0,0,10.0,1,"{""Casual"": 162, ""alternative_game_mode"": 65}",Casual,163,2,Percentage,0,...,1,0,0,0,0,0,0,0,1,0


***daysSinceInstall*** --> does not give any signal so I passed this one.

In [221]:
df.daysSinceInstall.sort_values(ascending=False).head()

927732    458.0
922612    457.0
906681    457.0
907975    457.0
907611    457.0
Name: daysSinceInstall, dtype: float64

***hasEnoughCoin*** --> bool, no work required.

***Thoughts: ***I will create a completely new dataframe including people who do not have enough coin, because if a player do not have enoughcoin and still pays with ingamecurrency coins to proceed that mean the player bought coins with actual currency. 

***hcl*** --> answer 

(1)how many causal and alternative modes players played. 

(2) if they played highly alternate event based game modes(christmas, haloween etc.)

In [222]:
df_test = df.copy()

In [223]:
df_test.columns

Index([    'Unnamed: 0.1',         'adOption', 'daysSinceInstall',
          'hasEnoughCoin',              'hcl',        'levelMode',
            'levelNumber',         'numFails',      'offerReason',
         'percentageLeft',         'platform',         'playTime',
               'playerId',         'segments',         'sessions',
                     'ts',      'target_both',   'target_adwatch',
          'target_bought',                  4,                  5,
                        6,               'w0',              'w10',
                    'w20',              'w30',              'w40',
                    'w50',              'w60'],
      dtype='object')

In [224]:
df['hcl'] = df['hcl'].map(convert_str_to_dict)

In [225]:
df['casual_mode'] = df['hcl']
df['alternative_mode'] = df['hcl']
df['event_mode'] = df['hcl']

df.drop(['hcl'], axis = 1, inplace = True)

In [226]:
type(df)

pandas.core.frame.DataFrame

In [227]:
def hcl_event_mode(x):
    if len(x) > 2:
        event_mode = 1
    else:
        event_mode = 0
    
    return event_mode

def hcl_casual(x):
    if 'Casual' in x:
        return x['Casual']
    else:
        return 0

def hcl_alternate(x):
    if 'alternative_game_mode' in x:
        return x['alternative_game_mode']
    else:
        return 0

df['casual_mode'] = df['casual_mode'].map(hcl_casual)
df['alternative_mode'] = df['alternative_mode'].map(hcl_alternate)
df['event_mode'] = df['event_mode'].map(hcl_event_mode)

***levelMode***

In [228]:
def levelMode(x):
    if x == 'Casual':
        return 1
    else:
        return 0

In [229]:
df.levelMode = df['levelMode'].map(levelMode)

In [230]:
df.levelMode.unique()

array([1, 0])

***levelNumber,numFails*** --> keep it as it is, yet feel free to use sklearn's standardscaler later on

***offerReason***

In [231]:
df.offerReason.unique()

array(['Percentage', 'LevelTime', 'BoosterUsed'], dtype=object)

In [232]:
df = one_hot_dummy(df,['offerReason'])

***platform***

In [233]:
df.platform.unique()

array(['IOS', 'Android'], dtype=object)

In [234]:
def platform(x):
    if x == 'IOS':
        return 1
    else:
        return 0

In [235]:
df.platform = df['platform'].map(platform)

***segments***

In [236]:
import json

In [237]:
df.segments = df.segments.map(convert_str_to_dict)

In [238]:
df.segments[0]

{'Country': 'SA',
 'IAP': 'false',
 'ISG_EnergyRefill': 'Low',
 'ISG_PBS_Price': 'Low',
 'ISG_SaveMe_Max_Percentage': 'Mid',
 'LanguageCode': 'en'}

In [239]:
def segments_Country(x):
    if 'Country' in x:
        return x['Country']


def segments_LanguageCode(x):
    if 'LanguageCode' in x:
        return x['LanguageCode']

def segments_IAP(x):
    if 'IAP' in x:
        if x['IAP'] == 'false':
            return False
        else:
            return True

df['country'] = df['segments'].map(segments_Country)
df['language'] = df['segments'].map(segments_LanguageCode)
df['paid_before'] = df['segments'].map(segments_IAP)

In [240]:
df.paid_before.value_counts()

False    941863
True      21729
Name: paid_before, dtype: int64

In [241]:
df.drop(['segments'], axis = 1, inplace = True)

***Country***

When I ran the country with dummy variables in a logistic regression model,the signal from most countries were weak. Thereby I decided to seperate countries based on avg disposable income tax per capita.

In [261]:
high_d_income = ['CH','MT','GL','AW','PF','AO','BS','NC','RE','BB','IS','FO','GD','MW','CI','BE','FR','FI','EU','SG','NL','AU','OM','DE','IL','SE','US','DK','ZA','CA','QA','GB','JP','KR','AT','IT','AE','AD','NZ']   #FIRST ~50 IN WORLD RANKING

In [262]:
def disposable_income_class(x):
    high_d_income = ['CH','MT','GL','LU','AW','PF','AO','BS','NC','RE','BB','IS','FO','GD','MW','CI','BE','FR','FI','EU','SG','NL','AU','OM','DE','IL','SE','US','DK','ZA','CA','QA','GB','JP','KR','AT','IT','AE','AD','NZ']   #FIRST ~50 IN WORLD RANKING
#     other_d_income = ['RU','MR','GU','ZW','CR','HT','SC','BM','MV','MP','SV','GN','CW','HN','KN','GY','BZ','PY','JE','ET','GM','FM','SN','TZ','ME','GH','LA','MV','','AF','AS','MN','MO','PE','TT','GE','BO','CO','VE','JM','UG','UY','BA','EE','KZ','SY','GI','CZ','PA','DO','PR','CN','MY','PL','TM','MK','EC','VN','PT','MX','HR','MQ','KE','NG','GT','LK','SI','KH','MD','IN','DZ','CL','SD','NP','AZ','HK','KW','IR','BG','CY','PS','AP','BD','AR','PK','MU','BN','AL','AX','SK','JO','RO','LB','LT','SA','BR','TH','HU','MA','YE','TR','IQ','UA','ID','ES','IE','LV','TJ','RU','BH','TW','LY','TN','KG','AM','EG','PH','UZ','NO','BY','RS','GR'] 
    
    if x in high_d_income:
        return 1
    else:
        return 0

df.country = df.country.map(disposable_income_class)

In [263]:
# df = one_hot_dummy(df,['country'])

In [264]:
df = one_hot_dummy(df,['language'])

***sessions***

In [267]:
max(df.sessions)

4057

In [268]:
min(df.sessions)

-99999

In [269]:
(df.sessions < 0).sum() / len(df.sessions)

0.0005998389359812037

In [270]:
def sessions(x):
    if x < 0:
        return 1
    else:
        return x

In [271]:
df.sessions = df['sessions'].map(sessions)

Save the dataframe.

In [282]:
df.to_csv('data_the_babe/df_1m_comp.csv')

In [283]:
print(df.columns)

Index([    'Unnamed: 0.1',         'adOption', 'daysSinceInstall',
          'hasEnoughCoin',        'levelMode',      'levelNumber',
               'numFails',   'percentageLeft',         'platform',
               'playTime',         'playerId',         'sessions',
            'target_both',   'target_adwatch',    'target_bought',
                        4,                  5,                  6,
                     'w0',              'w10',              'w20',
                    'w30',              'w40',              'w50',
                    'w60',      'casual_mode', 'alternative_mode',
             'event_mode',      'BoosterUsed',        'LevelTime',
             'Percentage',          'country',      'paid_before',
                'Unknown',               'de',               'en',
                     'es',               'fr',               'it',
                  'pt-BR',               'ru',               'tr',
                  'zh-CN'],
      dtype='object')
